In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import re
from nltk.util import ngrams

In [2]:

from sentence_transformers import SentenceTransformer
from InstructorEmbedding import INSTRUCTOR
model_1 = INSTRUCTOR('moka-ai/m3e-base')


load INSTRUCTOR_Transformer
max_seq_length  512


In [3]:
def generate_ngrams(row, n):
    
    res= []
    
    if pd.isnull(row['评论内容']) == False:
    
        ngramss = ngrams( re.split('，|。|！|？', row['评论内容']), n)

        for grams in ngramss:
            if grams != '':
                tmp = ''
                for k in range(0, n):
                    tmp = tmp + grams[k] + '，'
                tmp = tmp[:-1]
                res.append(tmp)
        
    return res
            

In [6]:
def changeno(df): # 其实没什么用
    n=1
    for k in range(0,len(df)):
        if k == 0:
            prev =  df['评论序号'].iloc[k]
            df.at[k,'no'] = n
        else:
            if prev == df['评论序号'].iloc[k]:
                df.at[k,'no'] = n
            else:
                prev = df['评论序号'].iloc[k] 
                n = n+1
                df.at[k,'no'] = n
    return (df)
        

In [11]:
# 使用moka-ai/m3e-base模型
####for query - n gram = 1
df = pd.read_excel('完整版评论分类1.xlsx', sheet_name = '深圳四季')
df = changeno(df)
df['评论内容_new'] = df.apply(generate_ngrams, axis=1,n=1)

df = df.explode('评论内容_new')
query_df = df[['评论序号','评论内容','评论内容_new', 'no']].dropna()
query_df['query_embedding'] = query_df['评论内容_new'].apply(lambda x: model_1.encode([x]))

query_df.to_pickle('query_df_final.pkl')
query_df


,评论序号,评论内容,评论内容_new,no,query_embedding
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,酒店挺不错的,1.0,"[[0.38339633, 0.6643063, 0.70754176, -0.102897..."
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,房间干净卫生,1.0,"[[-0.18294388, 0.45822287, 0.27038223, -0.9868..."
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,设备齐全,1.0,"[[0.26044473, 0.67715794, 0.4333286, -0.249953..."
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,床品舒适,1.0,"[[0.2529874, 0.5675511, -0.035674542, 0.109394..."
0,1,酒店挺不错的，房间干净卫生，设备齐全，床品舒适，早餐精致好吃。,早餐精致好吃,1.0,"[[0.21657318, 0.9036295, 0.22838418, -0.464586..."
...,...,...,...,...,...
998,999,酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便...,酒店位置很不错,999.0,"[[0.08939012, 0.70096344, 0.8776309, 0.0753267..."
998,999,酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便...,地铁就在楼下,999.0,"[[-0.040009186, 0.14445141, 0.98486614, 0.2817..."
998,999,酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便...,自驾车到酒店,999.0,"[[0.26780212, 0.25090283, 0.7855235, -0.187996..."
998,999,酒店位置很不错，地铁就在楼下，自驾车到酒店，换乘地铁出行都很方便...,换乘地铁出行都很方便...,999.0,"[[-0.20931117, 0.49199355, 0.5363215, -0.46087..."


In [12]:
# 使用moka-ai/m3e-base模型
#### faq
df = pd.read_excel('faq(new).xlsx')
df['faq_new'] = df['分类结果'] + ':' + df['分类关键句']
df['faq_embedding'] = df['faq_new'].apply(lambda x: model_1.encode([x]))
df.to_pickle('faq_df_final.pkl')
df

,分类结果,分类关键句,faq_new,faq_embedding
0,房间-布局-不合理,一开始的房间风水怪怪的,房间-布局-不合理:一开始的房间风水怪怪的,"[[0.41805333, 0.42092174, 0.66800374, -0.09395..."
1,房间-布局-不合理,房间格局奇怪的不得了，一根大柱子在房间,房间-布局-不合理:房间格局奇怪的不得了，一根大柱子在房间,"[[0.3116825, 0.6138913, 0.69666356, -0.0623900..."
2,房间-布局-不合理,洗手间倒是挺大,房间-布局-不合理:洗手间倒是挺大,"[[0.24111979, 0.7401576, 0.7612051, -0.0776110..."
3,房间-布局-合理,老五星 品质还是在的；房间格局很好,房间-布局-合理:老五星 品质还是在的；房间格局很好,"[[0.8829135, 0.605288, 0.96933293, -0.27354595..."
4,房间-大小-合适,房间够大,房间-大小-合适:房间够大,"[[-0.08642605, 0.25674045, 0.71628565, -0.5995..."
...,...,...,...,...
1240,位置-周围环境-优美,环境不错，很干净，离火车站非常近,位置-周围环境-优美:环境不错，很干净，离火车站非常近,"[[-0.25988665, 0.95085675, 0.7038429, -0.20231..."
1241,位置-周围环境-优美,环境优雅，方便。该装修了,位置-周围环境-优美:环境优雅，方便。该装修了,"[[-0.33010948, 0.79241335, 0.518667, -0.215956..."
1242,位置-周围环境-优美,地理位置优越,位置-周围环境-优美:地理位置优越,"[[-0.026185513, 0.99884313, 1.2126979, 0.26789..."
1243,位置-周围环境-优美,地理位置优越，美丽的酒店,位置-周围环境-优美:地理位置优越，美丽的酒店,"[[-0.121659584, 0.70647943, 0.8320463, -0.0708..."
